# Character-Level Tokenizing

Thoughts

* Unlike word-based vocabulary size, character-based will be 1-2 orders smaller: tens to hundreds (ascii- 122-ish) outside of some logographic and syllabaries.
* Thus, follows there will be fewer out-of-vocabulary tokens as well. 
* Significantly less information (context) dense. 
* Done A. Karpaty's NanoGPT? that uses character-level tokenization 
* Downside: strings get transtated into significantly longer sequences for processing. this means a shorter context can be passed around


# Imports

In [1]:
%load_ext kedro.ipython
%reload_kedro

from typing import Any, Dict, List, Tuple

import re

from datasets import load_dataset
from datasets.dataset_dict import DatasetDict
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from transformers import CanineTokenizer

[02/21/24 13:10:35] INFO     Resolved project path as: /Users/b/Documents/ML/tolkenizers.           ]8;id=981502;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=936932;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#149\149]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/21/24 13:10:36] INFO     Kedro project tolkenizers                                              ]8;id=42383;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=766364;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#119\119]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=15094;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=820395;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#120\120]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=334340;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=621821;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#126\126]8;;\

                    INFO     Resolved project path as: /Users/b/Documents/ML/tolkenizers.           ]8;id=920863;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=877332;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#149\149]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Kedro project tolkenizers                                              ]8;id=820071;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=869225;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#119\119]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=945302;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=123612;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#120\120]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=833821;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=766793;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#126\126]8;;\

                    WARNING  /Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/ ]8;id=192747;file:///Users/b/.pyenv/versions/3.11.7/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=51816;file:///Users/b/.pyenv/versions/3.11.7/lib/python3.11/warnings.py#109\109]8;;\
                             tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update                      
                             jupyter and ipywidgets. See                                                           
                             https://ipywidgets.readthedocs.io/en/stable/user_install.html                         
                               from .autonotebook import tqdm as notebook_tqdm                                     
                                                                                                                   

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Data 

In [2]:
shakespeare = load_dataset("tiny_shakespeare")
train_split = shakespeare["train"]
test_split = shakespeare["test"]

# Implementation

Take your corpus, and split on every (ascii, unicode, ..) character. that's it

In [3]:
s = shakespeare['train']['text'][0][:100]

def char_split(l) -> list[str]:
    chars = []
    for s in l: 
        chars.extend([c for c in s])
    return chars

print([s])
print(char_split(s))

['First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou']
['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B', 'e', 'f', 'o', 'r', 'e', ' ', 'w', 'e', ' ', 'p', 'r', 'o', 'c', 'e', 'e', 'd', ' ', 'a', 'n', 'y', ' ', 'f', 'u', 'r', 't', 'h', 'e', 'r', ',', ' ', 'h', 'e', 'a', 'r', ' ', 'm', 'e', ' ', 's', 'p', 'e', 'a', 'k', '.', '\n', '\n', 'A', 'l', 'l', ':', '\n', 'S', 'p', 'e', 'a', 'k', ',', ' ', 's', 'p', 'e', 'a', 'k', '.', '\n', '\n', 'F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'Y', 'o', 'u']


## Canine Tokenizer

In [4]:
s = shakespeare['train']['text'][:100]

tokenizer = CanineTokenizer.from_pretrained("google/canine-c")

In [5]:
encoding = tokenizer(s, padding="longest", truncation=True)
encoding


{
    'input_ids': [
        [
            57344,
            70,
            105,
            114,
            115,
            116,
            32,
            67,
            105,
            116,
            105,
            122,
            101,
            110,
            58,
            10,
            66,
            101,
            102,
            111,
            114,
            101,
            32,
            119,
            101,
            32,
            112,
            114,
            111,
            99,
            101,
            101,
            100,
            32,
            97,
            110,
            121,
            32,
            102,
            117,
            114,
            116,
            104,
            101,
            114,
            44,
            32,
            104,
            101,
            97,
            114,
            32,
            109,
            101,
            32,
            115,
            112,
            10